In [ ]:
# Import dependencies
from PIL import Image, ImageDraw, ImageFont
import requests
from zipfile import ZipFile, ZIP_DEFLATED
import shutil
from io import BytesIO
import os

# 1. Google Maps layer data gathering

In [5]:
# Project URL must be int this format: https://www.google.com/maps/d/edit?mid=...&usp=sharing
project_url = YOUR_PROJECT_URL

# Extract id from project URL
id = project_url.replace("&usp=", "?mid=").split("?mid=")[1]

In [6]:
url = "https://www.google.com/maps/d/kml?mid=" + id

# Download project files
response = requests.get(url)

In [7]:
# Save the respone as .kmz file
kmz_file_name = "response.kmz"
with open(kmz_file_name, mode="wb") as file:
  file.write(response.content)

In [8]:
# Extract .kmz file content to content folder
kml_file_name = "response.kml"
# .kmz to .kml conversion
with ZipFile(kmz_file_name) as kmzFile:
    # extract files
    kmzFile.extractall("content")

current_path = os.getcwd()

# Remove old pictures files, to correct any data errors
shutil.rmtree(f"{current_path}/content/images")
os.remove(f"{current_path}/response.kmz")


In [9]:
import xml.etree.ElementTree as ET

# Load the .kml file
tree = ET.parse(f"{current_path}/content/doc.kml")
root = tree.getroot()

# Namespace to account for XML tags
namespace = '{http://www.opengis.net/kml/2.2}'

# This is done for the purpose of displaying red markers on top of other layers
data_correction = {
  "0": "0-red",
  "1": "2-blue",
  "2": "3-green",
  "3": "4-yellow",
  "4": "1-white",
}

data_count = {
  "0-red": 0,
  "1-white": 0,
  "2-blue": 0,
  "3-green": 0,
  "4-yellow": 0,
}

# Iterate through all the elements to find Folder tags
for elem in root.iter():

    # Layers are represented with Folder tag
    if elem.tag.endswith('Folder'):
        # Find the folder name
        folder_name = elem.find(f'{namespace}name')

        if folder_name is not None:
            # Print folder name for verification
            print(f'Folder Name: {folder_name.text}')

            # Create a new KML structure with just this folder
            new_kml = ET.Element('kml', xmlns="http://www.opengis.net/kml/2.2")
            document = ET.SubElement(new_kml, 'Document')  # Add a Document tag
            folder_copy = ET.SubElement(document, 'Folder')  # Copy the Folder element into the new document

            folder_nr = folder_name.text.strip().split(" - ")[0]

            # Ignore data inside 5
            if folder_nr != "5":

              # Folder element name
              current_folder_name = folder_copy.find(f'{namespace}name')

              # Renaming it with our naming convention
              if current_folder_name is not None:
                current_folder_name.text = data_correction[folder_nr]

              child_count = -1
              for child in elem:
                child_count += 1

                if child.tag == f"{namespace}Placemark":
                  name_element = child.find(f'{namespace}name')
                  if name_element is not None:
                    name_element.text =  data_correction[folder_nr] + "-" + str(child_count).zfill(5)
                  else:
                    print("Warning: <name> tag not found in Placemark")

                # Append the modified (or unmodified) child to the folder_copy
                folder_copy.append(child)

              # Save the counts for image creation
              data_count[data_correction[folder_nr]] = child_count

              # Create new folder
              new_folder_name = f"{current_path}/content/{data_correction[folder_nr]}"
              os.makedirs(new_folder_name, exist_ok=True)


              # Save the new KML file with the folder's name
              new_kml_tree = ET.ElementTree(new_kml)
              new_filename = f"{current_path}/content/{data_correction[folder_nr]}/{data_correction[folder_nr]}.kml"
              new_kml_tree.write(new_filename)
              print(f'Saved new KML file: {new_filename}')

# Remove unneccesary files
os.remove(f"{current_path}/content/doc.kml")

Folder Name: 0 - Maantee
Saved new KML file: /content/content/0-red/0-red.kml
Folder Name: 1 - Linn
Saved new KML file: /content/content/2-blue/2-blue.kml
Folder Name: 2 - B teed
Saved new KML file: /content/content/3-green/3-green.kml
Folder Name: 3 - kruus
Saved new KML file: /content/content/4-yellow/4-yellow.kml
Folder Name: 4 - Op-ala
Saved new KML file: /content/content/1-white/1-white.kml
Folder Name: 5 - Autod


# Generate spot images

In [10]:
font_url = "https://raw.githubusercontent.com/tsenart/sight/master/fonts/Consolas.ttf"

# Fetch the font file from the URL
response = requests.get(font_url)
font_height = 26
font = ImageFont.truetype(BytesIO(response.content), font_height)


In [11]:
img_size = 32, 32
text_size = 20,10

# Spot colours
blue = (60, 160, 230)
red = (145, 40, 40)
white = (255, 255, 255)
yellow = (255, 255, 30)
green = (55, 165, 50)

local_path = f"{current_path}/content"

In [12]:
def drawImage(number, img_name, color, path):
  H = 32

  char_width = 0.55
  number_length = len(str(abs(number)))
  W = round(font_height*number_length*char_width) + 8

  img = Image.new('RGB', (W,H), (0, 0, 0, 0))
  draw = ImageDraw.Draw(img)
  draw.rounded_rectangle((0, 0, W-1, H-1), fill=color, outline=(255, 255, 255), width=1, radius=4)

  if color == "white" or color == "yellow":
    draw.text((4, 4), str(number),font=font, fill=(0,0,0))
  else:
    draw.text((4, 4), str(number),font=font, fill=(255,255,255))

  img.save(os.path.join(img_dir_path, str(img_name) + '.png'))


In [13]:
for dir_name in os.listdir(local_path):
  dir_path = os.path.join(local_path, dir_name)

  img_dir_path = os.path.join(dir_path, "images")


  os.makedirs(img_dir_path, exist_ok=True)

  img_count = data_count[dir_name]
  color = dir_name.split("-")[1]

  for i in range(img_count):
    img_name = dir_name + "-" + str(i+1).zfill(5)
    drawImage(i+1, img_name, color, img_dir_path)


In [ ]:
#shutil.rmtree(f"{current_path}/content/.ipynb_checkpoints")
shutil.rmtree(f"{current_path}//output/")
#shutil.rmtree(f"{current_path}/sample_data")

# Pack the files to .kmz format

In [14]:
response_path = '/content/output'
# Make sure the response dir exist
os.makedirs(response_path, exist_ok=True)

for dir_name in os.listdir(local_path):
  dir_path = os.path.join(local_path, dir_name)

  kmz_filename = f"{dir_name}.kmz"
  kmz_file_path = os.path.join(response_path, kmz_filename)

  with ZipFile(kmz_file_path, 'w', compression=ZIP_DEFLATED) as kmz:
    for root, dirs, files in os.walk(dir_path):
        for file in files:
            file_path = os.path.join(root, file)

            # Use arcname to maintain the directory structure inside the KMZ
            arcname = os.path.relpath(file_path, dir_path)
            kmz.write(file_path, arcname=arcname)

shutil.rmtree(f"{current_path}/content/")